In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/28 06:10:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
df_green.registerTempTable('green')

/home/apasek/spark/spark-3.3.1-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [5]:
df_green_revenue = spark.sql('''
    SELECT 
        DATE_TRUNC('HOUR', lpep_pickup_datetime) AS hour, 
        PULocationID AS zone,
        SUM(total_amount) AS amount,
        COUNT(*) AS number_records
    FROM green
    WHERE lpep_pickup_datetime >= '2020-01-01'
    GROUP BY 1,2
''')

In [6]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite') 

In [7]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [8]:
df_yellow.registerTempTable('yellow')

In [9]:
df_yellow_revenue = spark.sql('''
    SELECT 
        DATE_TRUNC('HOUR', tpep_pickup_datetime) AS hour, 
        PULocationID AS zone,
        SUM(total_amount) AS amount,
        COUNT(*) AS number_records
    FROM yellow
    WHERE tpep_pickup_datetime >= '2020-01-01'
    GROUP BY 1,2
''')

In [10]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite') 

In [11]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [12]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [13]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|   7| 769.7299999999996|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|  12|              null|                null|             107.0|                    6|
|2020-01-01 00:00:00|  37|            175.67|                   6|161.60999999999999|                    7|
|2020-01-01 00:00:00|  40|168.97999999999996|                   8|             89.97|                    5|
|2020-01-01 00:00:00|  45|  

In [14]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')